<a href="https://colab.research.google.com/github/shyoonCS/DataAnalysis/blob/main/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EA%B3%BC%EC%A0%9C(%EC%BD%94%EB%93%9C%2C_%EA%B3%BC%EC%A0%9C%EC%88%98%ED%96%89%EB%B3%B4%EA%B3%A0%EC%84%9C)_202534_364010(%EC%9C%A4%EC%84%A0%ED%9D%AC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###머신러닝 출석과제 [K-근접이웃 분류기 구현하기]
* 학  번 : 202534-364010
* 성  명 : 윤선희
---



##<span style="color:blue"> **1. 코드, 코드설명(주석)**</span>

In [1]:
# 수행에 필요한 라이브러리 불러오기
import csv           # csv 파일 읽기 위한 라이브러리
import random        # 데이터 랜덤 처리 라이브러리 : 난수 생성, 리스트의 데이터를 랜덤으로 순서 변경 또는 데이터 추출
import math          # 수학함수 제공
import pandas as pd  # DataFrame, Series 데이터 객체 사용

(1) 데이터 불러오기

In [2]:
# CSV 파일을 읽어서 리스트 형태로 저장, 데이터 구성 : # float형 특성값 4개, 정수형 레이블 1개(class)
iris_data = []
with open("iris_KNN.csv", "r", encoding="utf-8") as f:
    fdata = csv.reader(f)
    for row in fdata:
        features = list(map(float, row[:4]))  # map함수를 이용하여 list 4개 값을 float 변환하여 featuers list 구성
        label = int(row[4])                   # class label
        iris_data.append((features, label))   #리스트 형태로 iris_data 구성 ex: ([5.0, 3.5, 1.6, 0.6], 1)

(2) 학습데이터, 테스트데이터 분할

In [3]:
 # 데이터 분할 : 학습 데이터와 테스트 데이터 분할
random.shuffle(iris_data)    # 리스트의 데이터를 랜덤으로 순서 변경
train_data = iris_data[:100] # 0~99까지 100개의 데이터를 뽑아 학습데이터로 할당
test_data = iris_data[100:]  # 100~ 나머지 데이터를 테스트데이터로 할당

(3) 거리 계산 함수(유클리디안 거리)
* **유클리디안 거리(Euclidean distance)** : 두 점 사이의 직선 거리를 계산하는 방식, 벡터간 유사도 계산에 활용


In [5]:
# 유클리디안 거리 계산 : 두 리스트의 각 요소를 튜플로 처리하여, 차이 제곱근 계산
def euclidean_distance(train_feature, test_feature) :
    return math.sqrt(sum((a - b) ** 2 for a, b in zip(train_feature, test_feature)))

(4) K-최근접이웃 분류기 구현
* **K-NN(K-Nearest Neighborhood)** : 새로운 데이터와 가장 가까운 K개의 이웃 데이터의 클래스를 모아,
다수결로 해당 데이터의 클래스를 결정하는 분류알고리즘(classfication, 지도학습)
* K값(하이퍼파라미터) 선택이 중요하며, 너무 작으면 노이즈에 민감, 너무 크면 구분력이 떨어질 수 있다.
* scikit-learn 알고리즘 : KNeighborsClassifier

In [6]:
def knn_predict(train, test_feature, k):
    # Step1. 학습 데이터와 테스트 데이터 사이의 거리를 계산
    distances = []
    for features, label in train:
        dist = euclidean_distance(features, test_feature)
        distances.append((dist, label))  # 거리와 해당 레이블을 튜플로 저장

    # step2. 거리를 기준으로 오름차순 정렬
    distances.sort(key=lambda x: x[0])

    # step3. 가장 가까운 k개의 데이터 선택
    k_nearest = distances[:k]

    # step4. k개의 데이터의 레이블 카운트
    label_count = {}
    for dist, label in k_nearest:
        if label in label_count:
            label_count[label] += 1  # 이미 있으면 1 증가
        else:
            label_count[label] = 1  # 없으면 1로 초기화

    # step5. 클래스 레이블 결정(다수결)
    predicted_label = max(label_count, key=label_count.get)  # value 기준 최대값
    return predicted_label


(5) K값에 따른 분류정확도(accuracy) 계산

In [7]:
results = []
for k in [5, 10, 20, 30]:       # K값 하이퍼파라미터 지정
    correct = 0
    for features, label in test_data:
        pred = knn_predict(train_data, features, k)
        if pred == label:       # test_data의 label과 일치한 경우 카운트
            correct += 1
    accuracy = correct / len(test_data) * 100 # 분류정확도(accuracy) % 계산
    results.append((k, accuracy))             # K값별 분류율 리스트 추가


##<span style="color:blue"> **2. 실험결과표**</span>

In [21]:
print("[ K값에 따른 분류 결과 ]")
df = pd.DataFrame(results, columns=["K값", "분류율(%)"])
from IPython.display import display, HTML # HTML 형식 출력
display(HTML(df.to_html()))               # DataFrame 결과를 HTML 표로 표현

[K값에 따른 분류 결과]


,K값,분류율(%)
0,5,94.0
1,10,92.0
2,20,88.0
3,30,90.0


##<span style="color:blue"> **3. 결과분석**</span>
(1) 실험 결과
* K값이 가장 작은 경우(K=5) 분류율이 가장 높고 K값이 큰 경우(K=20) 분류율이 가장 낮았다.
* KNN 알고리즘 특성상 하이퍼파라미터 K 값에 따라 영향을 받는다.
* K가 작으면 훈련 데이터에 민감하여 과적합 우려, 노이즈 영향이 크고,
* K가 크면 노이즈에는 강하나 구분력 저하, 과소적합 우려가 있다.

(2) KNN 성능 향상을 위한 고려 사항
* **최적 K값 선정 방법** : <span style="color:blue">교차검증(Cross Validation)</span> 을 수행하여 검증 정확도가 가장 높은 K를 선택한다.
* **거리 알고리즘 선택** : 데이터 특성(연속형, 범주형, 이상치 유무)에 따라 선택한다.
  + 연속형 : 유클리디안(euclidean), 맨해튼 거리(manhattan, 격자계산)
  + 범주형 : 해밍 거리(hamming)
  + 상관관계 고려 : 마할라노비스(mahalanobis, 데이터 분산 고려)